In [12]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from torch.autograd import Variable
from torchvision import datasets, models, transforms, utils

import torch.nn.functional as F
import os, glob

import torch.optim as optim
from torch.optim import lr_scheduler

In [13]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
base_dir = '/Users/jooyoungson/Datasets/Fashion'

In [14]:
class_num = glob.glob(base_dir + '/train/**')
len(class_num)

39

In [15]:
trans = transforms.Compose([transforms.Resize((224, 224)), 
                            transforms.ToTensor(), 
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) #img Tensor로 변경후 normalize

train_dataset = torchvision.datasets.ImageFolder(root=base_dir +'/train', 
                                              transform = trans )
val_dataset = torchvision.datasets.ImageFolder(root=base_dir + '/val', transform = trans)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

In [16]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/Users/jooyoungson/Datasets/Fashion'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        str_epoch = str(epoch)
        torch.save(model.state_dict(), data_dir+"model-"+str_epoch+".pth") # Save checkpoint

        input_tensor = torch.rand(1, 3, 224, 224) #an example input
        script_module = torch.jit.trace(model, input_tensor)
        script_module.save(data_dir+"class1.pt")

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    

    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
model_ft = models.mobilenet_v2(pretrained=True)
# backbone = model_ft.features
# new_model = nn.Sequential(backbone, nn.AdaptiveAvgPool2d(1000), nn.Linear(1000,39))

model_ft.classifier[1].out_features = 39